## Apoyo a la explicación de algunos conceptos del tema 4 (Procesos de Decisión de Markov)

In [1]:
import random

Estas definiciones Python permiten explicar, con un ejemplo, el concepto de valoración de un estado respecto de de una política dada (notado $V^{\pi}(e)$), en el contexto de los Procesos de Decisión de Markov. 

Algunas de las definiciones que aquí veremos se solapan con el principio de la Práctica 4. 

Supondremos que un Procesos de Decisión de Markov (MDP, por sus siglas en inglés) va a ser un objeto de la siguiente clase (o mejor dicho, de una subclase de la siguiente clase). 

In [2]:
class MDP(object):

    """La clase genérica MDP tiene como métodos la función de recompensa R,
    la función A que da la lista de acciones aplicables a un estado, y la
    función T que implementa el modelo de transición. Para cada estado y
    acción aplicable al estado, la función T devuelve una lista de pares
    (ei,pi) que describe los posibles estados ei que se pueden obtener al
    plical la acción al estado, junto con la probabilidad pi de que esto
    ocurra. El constructor de la clase recibe la lista de estados posibles y
    el factor de descuento.

    En esta clase genérica, las funciones R, A y T aparecen sin definir. Un
    MDP concreto va a ser un objeto de una subclase de esta clase MDP, en la
    que se definirán de manera concreta estas tres funciones"""  

    def __init__(self,estados,descuento):

        self.estados=estados
        self.descuento=descuento

    def R(self,estado):
        pass

    def A(self,estado):
        pass
        
    def T(self,estado,accion):
        pass

Consideramos el siguiente problema del robot en la cuadrícula, descrito en la diapositiva 7 del tema 4. Lo que sigue es una definición de los estados, acciones y recompensa para ese problema en una clase python. **No entraremos a dar los detalles sobre la siguiente implementación**, pero baste decir que:


* El constructor de la clase recibe la cuadrícula como una lista de listas de strings
* Los atributos `estados` y `recompensa` contienen la lista de estados y la recompensa. A diferencia de las coordenadas usadas en las diapositivas, un estado vendrá referencias por el par (f,c), con la fila y columna en la que se encuentra (de arriba a abajo, de izquierda a derecha y comenzando en 0). 
* La recompensa de un estado la da el método `R`.
* Las acciones aplicables a un estado se obtienen con el método `A`.
* Los estados y las probabilidades que resultan de aplicar una acción a un estado, se obtienen con el método `T`

In [3]:
cuadrícula_1 = [[' ',' ',' ',+1],
                [' ','*',' ',-1],
                [' ',' ',' ',' ']]


class Cuadrícula(MDP):
    """Supondremos que cuadrícula viene dada por una lista de listas, en la
    que si hay un número es la recompensa de esa casilla, si hay otra cosa la
    recompesa es la que se indica por defecto y si hay None es una casilla
    bloqueada. Supondremos que está bien construida"""

    def __init__(self,cuadrícula,recompensa_por_defecto=-0.04,descuento=0.9,ruido=0.2):

        estados=[]
        terminales=[]
        recompensa={}
        nfilas=len(cuadrícula)
        ncolumnas=len(cuadrícula[0])
        for i in range(nfilas):
            for j in range(ncolumnas):
                contenido=cuadrícula[i][j]
                if contenido != "*":
                    estados.append((i,j))
                    if isinstance(contenido,(int,float)):
                        recompensa[(i,j)]=contenido
                        terminales.append((i,j))
                    else:
                        recompensa[(i,j)]=recompensa_por_defecto
                        if contenido=='S':
                            estado_inicial=(i,j)
                    
        super().__init__(estados,descuento)
        self.estados_terminales=terminales
        self.nfilas=nfilas
        self.ncolumnas=ncolumnas
        self.recompensa=recompensa
        self.ruido=ruido
        self.desplazamientos={"arriba":[(-1,0),(0,-1),(0,1)],
                                         "abajo":[(1,0),(0,-1),(0,1)],
                                         "izquierda":[(0,-1),(-1,0),(1,0)],
                                         "derecha":[(0,1),(-1,0),(1,0)]}


    def R(self,estado):
        return self.recompensa[estado]

    def A(self,estado):
        if estado in self.estados_terminales:
            return ["exit"]
        else:
            return ["arriba","abajo","izquierda","derecha"]

    def T(self,estado,acción):

        def desplaza(estado,despl):
            x,y=estado
            i,j=despl
            nx,ny=x+i,y+j
            if (nx,ny) in self.estados:
                return nx,ny
            else:
                return x,y

        if acción=="exit":
            return([(estado,0)])
        else:
            despl=self.desplazamientos[acción]
            pok=1-self.ruido
            pnook=self.ruido/2
            return [(desplaza(estado,despl[0]),pok),
                      (desplaza(estado,despl[1]),pnook),
                      (desplaza(estado,despl[2]),pnook)]
            


            
            
            
cuad1_MDP=Cuadrícula(cuadrícula_1,descuento=0.8)

Algunos ejemplos de los elementos que componen este MDP:

In [4]:
cuad1_MDP.estados

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3)]

In [5]:
cuad1_MDP.estados_terminales

[(0, 3), (1, 3)]

In [6]:
cuad1_MDP.R((0,3))

1

In [7]:
cuad1_MDP.R((1,3))

-1

In [8]:
cuad1_MDP.R((2,1))

-0.04

In [9]:
cuad1_MDP.A((0,3))

['exit']

In [10]:
cuad1_MDP.A((1,3))

['exit']

In [11]:
cuad1_MDP.A((0,2))

['arriba', 'abajo', 'izquierda', 'derecha']

In [12]:
cuad1_MDP.T((2,2),"izquierda")

[((2, 1), 0.8), ((1, 2), 0.1), ((2, 2), 0.1)]

In [13]:
cuad1_MDP.T((2,1),"izquierda")

[((2, 0), 0.8), ((2, 1), 0.1), ((2, 1), 0.1)]

En general, dado un MDP, representaremos **una política** para el mismo como un diccionario cuyas claves son los estados, y los valores las acciones. Una política representa una manera concreta de decidir en cada estado la acción (de entre las aplicables a ese estado) que ha de aplicarse. 

Lo que sigue es nuestra representación de la política que aparece en el dibujo (a) de la izquierdo de la diapositiva 11 del tema 4.

In [14]:
pi1={(0,0):"derecha",
    (0,1):"derecha",
     (0,2):"derecha",
     (0,3):"exit",
     (1,0):"arriba",
     (1,2):"arriba",
     (1,3):"exit",
     (2,0):"arriba",
     (2,1):"izquierda",
     (2,2):"izquierda",
     (2,3):"izquierda"}   

Dado un MDP, un estado de partida, y una política concreta, podemos generar (muestrear) una secuencia de estados por los que iríamos pasando si vamos aplicando las acciones que nos indica la política: dado un estado de la secuencia, aplicamos a ese estado la acción que indique la política, y obtenemos un estado siguiente de manera aleatoria, pero siguiendo la distribución de probabilidad que indica el modelo de transición dado por el método T.  

La siguiente función `genera_secuencia_estados(mdp,pi,e,n)` devuelve una secuencia de estados de longitud n (menor si se llega antes a un estado terminal), obtenida siguiendo el método anterior. Aquí mdp es objeto de la clase MDP, pi es una política, e un estado de partida y n la longitud de la secuencia. 

In [15]:
# distr es un lista de pares (vi,pi) con los diferentes valores de la v.a. y
# sus probabilidades. 
def muestreo(distr):
    r=random.random()
    acum=0
    for v,p in distr:
        acum+=p
        if acum>r:
            return v
    
        
        
def genera_secuencia_estados(mdp,pi,e,n):
    actual=e
    seq=[actual]
    for _ in range(n-1):
        if actual in mdp.estados_terminales:
            break
        actual=muestreo(mdp.T(actual,pi[actual]))
        seq.append(actual)
    return seq


In [32]:
muestreo([((2, 1), 0.8), ((1, 2), 0.1), ((2, 2), 0.1)])

(2, 1)

Vemos a continuación algunos ejemplo de uso. Está claro que partiendo desde el mismo estado y usando siempre la misma política no tiene por qué salir siempre la misma secuencia, debido a la incertidumbre que se tiene:

In [33]:
genera_secuencia_estados(cuad1_MDP,pi1,(2,2),15)

[(2, 2),
 (2, 1),
 (2, 0),
 (1, 0),
 (1, 0),
 (0, 0),
 (0, 1),
 (0, 2),
 (0, 2),
 (0, 3)]

In [34]:
genera_secuencia_estados(cuad1_MDP,pi1,(2,2),15)

[(2, 2),
 (2, 1),
 (2, 1),
 (2, 0),
 (1, 0),
 (1, 0),
 (0, 0),
 (0, 0),
 (0, 1),
 (0, 2),
 (0, 3)]

In [36]:
genera_secuencia_estados(cuad1_MDP,pi1,(2,2),15)

[(2, 2), (2, 1), (2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3)]

Dado un MDP y una secuencia de estados, valoramos dicha secuencia como la suma de las recompensas de los estados de la secuencias (cada una con el correspondiente descuento). Se pide definir una función `valora_secuencia(seq,mdp)` que calcula esta valoración.

In [37]:
def valora_secuencia(seq,mdp):
    return sum(mdp.R(e)*(mdp.descuento**i) for i,e in enumerate(seq))
   

In [38]:
valora_secuencia([(2, 2), (2, 1), (2, 0), (1, 0), (0, 0), (0, 0), (0, 1), (0, 2), (0, 3)],cuad1_MDP)

# Resultado:
# 0.001326592000000043

0.001326592000000043

In [39]:
valora_secuencia([(2, 2), (1, 2), (0, 2), (0, 3)],cuad1_MDP)
# Resultado:
# 0.4144000000000001

0.4144000000000001

In [40]:
valora_secuencia([(2, 2),(2, 2),(2, 2),(2, 1),(2, 1),(2, 1),(2, 0),
                 (1, 0),(1, 0),(0, 0),(0, 1),(0, 2),(0, 3)],cuad1_MDP)

# Resultado:
# -0.11753662791680003

-0.11753662791680003

Puesto que comenzando en un estado dado y aplicando una política dada, uno puede obtener distintas secuencias, está claro que para definir el valor que espero obtener a partir de ese estado, debemos recurrir a la noción de valor medio esperado, que es lo que se explica acontinuación. 

Dada una política $\pi$, la valoración de un estado e respecto de esa política, $V^{\pi}(e)$, se define como la media esperada de las valoraciones de las secuencias que se pueden generar teniendo dicho estado como estado de partida. 

Usando las funciones anteriores, se puede define una función `estima_valor(e,pi,mdp,m,n)` que realiza una estimación aproximada del valor $V^{\pi}(e)$: para ello genera $n$ secuencias de tamaño $m$, y calcula la media de sus valoraciones.  

In [41]:
# Solución:

def estima_valor(e,pi,mdp,m,n):
    return (sum(valora_secuencia(genera_secuencia_estados(mdp,pi,e,m),mdp) 
                for _ in range(n)))/n

In [43]:
estima_valor((0,0),pi1,cuad1_MDP,15,100000)

0.30073632202017475

In [44]:
estima_valor((2,2),pi1,cuad1_MDP,15,100000)

-0.003170949481726171

In [45]:
estima_valor((0,2),pi1,cuad1_MDP,15,100000)

0.6828934146957003

In [46]:
estima_valor((2,3),pi1,cuad1_MDP,15,100000)

-0.13225104164815682

In [48]:
estima_valor((1,2),pi1,cuad1_MDP,20,200000)

0.34329887591050245

En las diapositivas, los cálculos que se hacen son con descuento $\gamma=1$. Comprobemos que los resultados del muestreo son aproximadamente los mismos que se muestran en la diapositiva 22:

In [49]:
cuad2_MDP=Cuadrícula(cuadrícula_1,descuento=1)

In [50]:
estima_valor((0,0),pi1,cuad2_MDP,15,100000)

0.8120495999997541

In [51]:
estima_valor((0,1),pi1,cuad2_MDP,15,100000)

0.8666859999989989

In [52]:
estima_valor((0,2),pi1,cuad2_MDP,15,100000)

0.9174860000009051

In [53]:
estima_valor((0,3),pi1,cuad2_MDP,15,100000)

1.0

In [54]:
estima_valor((1,0),pi1,cuad2_MDP,15,100000)

0.7618403999995726

In [55]:
estima_valor((1,2),pi1,cuad2_MDP,15,100000)

0.6617419999988575

In [56]:
estima_valor((1,3),pi1,cuad2_MDP,15,100000)

-1.0

In [57]:
estima_valor((2,0),pi1,cuad2_MDP,15,100000)

0.7021616000005418

In [58]:
estima_valor((2,1),pi1,cuad2_MDP,15,100000)

0.6490808000002289

In [59]:
estima_valor((2,2),pi1,cuad2_MDP,15,100000)

0.5979283999998648

In [60]:
estima_valor((2,3),pi1,cuad2_MDP,20,200000)

0.3866127999996091

Hemos calculado $V^{\pi}(e)$ mediante una función que ha simulado las secuencias de estados en un MDP, y haciendo la correspondiente media. Pero en las diapositivas del tema veremos que se puede calcular $V^{\pi}(e)$ resolviendo un sistema de ecuaciones lineales.   

Obsérvese que la función de valoración $V^{\pi}$ nos puede servir para decidir cuándo una política es mejor que otra: bastará comparar las valoraciones que una política y otra asignana a cada estado.

Definamos una nueva política `pi2` que en las dos filas superiores es igual que `pi1` pero que en los estados de la fila inferior aplica la acción que aparentemente le acerca más al estado de recompensa +1 (el estado `(0,3)`), obviando el riesgo que tendría al tener que pasar cerca del estado de recompensa -1 (el estado `(1,3)`).

In [61]:
pi2={(0,0):"derecha",
    (0,1):"derecha",
     (0,2):"derecha",
     (0,3):"exit",
     (1,0):"arriba",
     (1,2):"arriba",
     (1,3):"exit",
     (2,0):"derecha",
     (2,1):"derecha",
     (2,2):"arriba",
     (2,3):"izquierda"}   

Comparemos ahora las valoraciones medias que se obtienen desde cada estado, usando la política `pi2`, respecto de las obtenidas con la política `pi1`:

In [62]:
estima_valor((0,0),pi2,cuad2_MDP,15,100000)
# Resultado para pi1: 0.8125059999997477

0.8116467999997451

In [63]:
estima_valor((0,1),pi2,cuad2_MDP,15,100000)
# Resultado para pi1: 0.8679967999989912

0.8681415999989952

In [64]:
estima_valor((0,2),pi2,cuad2_MDP,15,100000)
# Resultado para pi1: 0.9177044000009043

0.9190280000009077

In [65]:
estima_valor((0,3),pi2,cuad2_MDP,15,100000)
# Resultado para pi1: 1.0

1.0

In [66]:
estima_valor((1,0),pi2,cuad2_MDP,15,100000)
# Resultado para pi1: 0.7617611999995648

0.7619871999995769

In [67]:
estima_valor((1,2),pi2,cuad2_MDP,15,100000)
# Resultado para pi1: 0.6598491999988518

0.6591195999988498

In [68]:
estima_valor((1,3),pi2,cuad2_MDP,15,100000)
# Resultado para pi1: -1.0

-1.0

In [70]:
estima_valor((2,0),pi2,cuad2_MDP,15,100000)
# Resultado para pi1: 0.7019868000005451

0.5054396000003352

In [72]:
estima_valor((2,1),pi2,cuad2_MDP,15,100000)
# Resultado para pi1: 0.6488920000002221

0.5241483999996479

In [73]:
estima_valor((2,2),pi2,cuad2_MDP,15,100000)
# Resultado para pi1: 0.5983319999998657

0.5745651999994105

In [74]:
estima_valor((2,3),pi2,cuad2_MDP,15,100000)
# Resultado para pi1: 0.36348279999980715

0.3574239999997888

Como se observa, la política `pi1` obtiene resultados mejores o iguales en todos los estados. Podemos deducir por tanto que `pi1` es mejor que `pi2` (de hecho `pi1` es mejor que culquier otra política, como se verá más adelante). 